## Hands on SVM
#### by MMA

In [1]:
import numpy as np
import cvxopt
import cvxopt.solvers
from scipy.io import loadmat 

#### Load Data

In [2]:
data = loadmat('dataset/scene_train.mat')
x_train = np.array(data['X'])
y_train = np.array(data['Y'][0])
y_train = y_train.astype(np.double)
N = x_train.shape[0]
C = 1

#### Creates P matrix

In [3]:
P = []
for i in range(N):
    row = []
    for j in range(N):
        row.append(y_train[i] * y_train[j] * \
                   np.matmul(x_train[i], np.transpose(x_train[j])))
    P.append(row)
P = cvxopt.matrix(P)

#### Creats q matrix

In [4]:
q = cvxopt.matrix(np.ones(N) * -1)

#### Creates G and h matrix for inequality matrix

In [5]:
tmp1 = np.diag(np.ones(N) * -1)
tmp2 = np.identity(N)
G = cvxopt.matrix(np.vstack((tmp1, tmp2)))

tmp1 = np.zeros(N)
tmp2 = np.ones(N) * C
h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

#### Creates A and b matrix for equality matrix

In [6]:
A = cvxopt.matrix(y_train, (1, N))
b = cvxopt.matrix(0.0)

#### Run QP solver

In [7]:
solution = cvxopt.solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0: -3.6090e+02 -3.6103e+03  2e+04  3e+00  6e-13


 1: -2.3391e+02 -2.0645e+03  3e+03  3e-01  4e-13


 2: -2.0295e+02 -7.7093e+02  8e+02  6e-02  3e-13


 3: -2.0708e+02 -3.9279e+02  2e+02  1e-02  3e-13


 4: -2.1961e+02 -2.9187e+02  8e+01  3e-03  3e-13


...

 8: -2.3511e+02 -2.3839e+02  3e+00  3e-15  3e-13


 9: -2.3590e+02 -2.3698e+02  1e+00  2e-15  3e-13


10: -2.3626e+02 -2.3644e+02  2e-01  4e-15  4e-13


11: -2.3633e+02 -2.3634e+02  6e-03  7e-15  4e-13


12: -2.3634e+02 -2.3634e+02  1e-04  3e-15  3e-13
Optimal solution found.


#### Derive Lagrange multipliers

In [8]:
a = np.ravel(solution['x'])

#### Support vectors have non zero lagrange multipliers

In [9]:
sv = a > 1e-5
ind = np.arange(len(a))[sv]
a = a[sv]
sv_x = x_train[sv]
sv_y = y_train[sv]

#### Derive weights and bias

In [10]:
w = np.zeros(x_train.shape[1])
for n in range(len(a)):
    w += a[n] * sv_y[n] * sv_x[n]
    
w0 = sv_y[0] - np.matmul(w, sv_x[0])

#### Run evaluation

In [11]:
test_data = loadmat('dataset/scene_test.mat')
x_test = np.array(test_data['X'])
y_test = np.array(test_data['Y'][0])
y_test = y_test.astype(np.double)

In [12]:
y_predict = np.sign(w0 + np.matmul(x_test, w))

In [113]:
tp = 0
fp = 0
fn = 0
for i in range(y_predict.shape[0]):
    if y_predict[i] == y_test[i] and y_test[i] == 1:
        tp += 1
    elif y_predict[i] != y_test[i] and y_predict[i] == 1:
        fp += 1
    elif y_predict[i] != y_test[i] and y_predict[i] == -1:
        fn += 1

precision = tp / (tp + fp)
recall = tp / (tp + fn)

print("Precision : ", precision)
print("Recall    : ", recall)

Precision :  0.09327548806941431
Recall    :  0.8431372549019608
